In [ ]:
import os,sys,inspect
from IPython.display import display, Markdown, Latex, Javascript, clear_output
import emoji
import time
import ipywidgets as widgets
import ipython_blocking
import numpy as np
import time
from tqdm import tqdm_notebook
import functools
import configparser
import webbrowser
import shutil

parentdir = os.path.abspath('../../../pymodules')

sys.path.insert(0,parentdir)
#from connect_device import connect
from connect_device_v2.vizualizate import vizulizated_finds_devices, advanced_search_device,DeviceBundle
from connect_device_v2.save_file import SaveFilesButton
from connect_device_v2.connect import connect_to_device
from connect_device_v2.find_device import FindDevice
from connect_device_v2.auxiliary_commands import check_error
import pandas as pd
from datetime import datetime
button = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )
testname="Время перестройки синтезаторов"
display(Markdown('### 💠 {}'.format(testname)))



In [ ]:
from connect_device_v2.find_device import FindDevice
box = FindDevice()
display(box)

In [ ]:
button = box.select_device_continue_button

In [ ]:
%blockrun button

In [ ]:
button1 = box.select_config_continue_button

In [ ]:
%blockrun button1

In [ ]:
inst = box.selected_device

In [ ]:
config =  box.config
device = config['DEVICE']
points = int(device['points'])
ifbw = int(device['filter_IF'])
band = int(device['band'])
synt_delay = float(device["syntDelay"])
att = int(device['athenuator'])
center = (device['center_point'].replace(" ", "")).split(',')
factor = float(device['factor'])
portCount= int(device['portCount'])

In [ ]:
date=datetime.now()
date=date.strftime('%Y-%m-%d %H:%M:%S')
device = inst.query("*IDN?")
device = device.replace(' ','')
device = device.split(',')
device_info = {'Время запуска':date,
              'device':device[0],
              'model':device[1],
              'serial':device[2],
              'version':device[3]}

In [ ]:
def get_setup():
    inst.write("SYST:PRES")
    inst.write("TRIG:SOUR BUS")
    check_error(inst)
    inst.write("INIT:CONT OFF")
    check_error(inst)
    inst.write("SENS:AVER ON")
    check_error(inst)
    inst.write("SENS:AVER:COUN 10")
    check_error(inst)
    inst.write(f"SENS:BAND {ifbw*10**6}")
    check_error(inst)
    inst.write(f"SENS:SWE:POIN {points}")
    check_error(inst)
    inst.write(f"SENS:FREQ:SPAN {int(band)*10**6}")
    check_error(inst)
    inst.write("SERV:RFCTL:POW:STAT 1")
    check_error(inst)
    inst.write(f"SERVice:RFCTL:POWer:ATT {att}")
    check_error(inst)
    inst.write("SERV:RFCTL:POW:DAC 6554")
    check_error(inst)    
    inst.write("CALC:MST 1")
    check_error(inst)


In [ ]:
def scan():
    inst.write("INIT")
    check_error(inst)
    inst.write("TRIG:SING")
    check_error(inst)
    inst.query("*OPC?")
    check_error(inst)

In [ ]:
def get_value(pbar):
    result={}
    max_time = 0
    for freq in center: 
        result[freq]={}
        inst.write(f"SENS:FREQ:CENT {int(freq)*10**6}")       
        check_error(inst)        
        first_sko = 0
        time_synt = 0
        for delay in np.arange (synt_delay, -0.5, -0.5):
            if time_synt != 0:
                result[freq][f'{delay}']= "-"                
            else:
                inst.write(f"SERVice:SYNThesizer:DELay:VALue {delay*10e-07}")
                check_error(inst)
                for i in range (10):
                    scan()
                normalizeited_graf(1)
                for i in range (10):
                    scan()
                scan()
                sko = float((inst.query("CALC:MST:DATA?").split(",")[1]))   
                check_error(inst)
                if delay == synt_delay:
                    first_sko = sko 
                elif sko / first_sko >= factor:
                    time_synt = (delay+0.5)
                result[freq][str(delay)] = sko
                first_sko = sko
        pbar.update(100/(len(center)*portCount))
        if time_synt == 0:
            result[freq]['time_synt'] =  "-"
        else:
            result[freq]['time_synt'] =  time_synt
            if time_synt > max_time:
                max_time = time_synt
        result['maximum'] =  max_time
    
    return result

In [ ]:
def get_measurement():
    pbar = tqdm_notebook(total=100)
    result={}
    for n in range(1,portCount+1):
        inst.write(f"CALC:PAR:DEF S{n}{n}")
        trace = inst.query("CALC:PAR:DEF?")
        result[f'{trace}']= get_value(pbar)
    return result
            

In [ ]:
def normalizeited_graf(n_graf):
    inst.write(f"CALC:PAR{n_graf}:SEL")
    check_error(inst)
    inst.write("CALC:MATH:MEM")
    check_error(inst)
    inst.write("CALC:MATH:FUNC DIVide")
    check_error(inst)

In [ ]:
get_setup()

In [ ]:
table = get_measurement()

In [ ]:
def table_generator(table):
    html_table='<table border="1" ><tr>' 
    html_table+="<td rowspan = '2'>Задержка, мкс</td>"
    for trace in table:        
            html_table+=f"<td colspan='{len(center)}'>СКО {trace} в полосе 100 МГц центральная частота, МГц</td>"
    html_table+= "</tr><tr>"
    for trace in table:
        for freq in table[trace]:
            if not freq.isdigit():
                continue
            html_table+=f"<td>{freq}</td>"


    for delay in np.arange (synt_delay, -0.5, -0.5):
        html_table+= f"</tr><tr><td>{delay}</td>"   
        for trace in table.keys():
            for freq in table[trace]: 
                if not freq.isdigit():
                    continue        
                html_table+=f"<td>{table[trace][freq][str(delay)]}</td>"


    html_table+= f"</tr><tr><td>Время установления:</td>"
    time_synt = 'time_synt'
    for trace in table:
        for freq in table[trace]:
            if not freq.isdigit():
                continue
            html_table+=f'<td>{table[trace][freq][time_synt]}</td>'


    html_table+= f"</tr><tr><td>Максимум:</td>"
    maximum = "maximum"
    for trace in table:
        html_table+=f"<td align = 'center', colspan= '{len(center)}' >{table[trace][maximum]}</td>"
    return html_table
        
        

             

In [ ]:
def html_header(data):
    html_table='<table border="1"><tr>'
    for key in data:
        html_table+=f'<td>{key}</td>'
    html_table+=f'</tr>'
    html_table+=f'<tr>'
    for key in data:
        html_table+=f'<td>{data[key]}</td>'
    html_table+=f'</tr>'    
    return html_table

In [ ]:
from IPython.core.display import display, HTML
html_table=table_generator(table)
header = html_header(device_info)


display(HTML(header),HTML(html_table))

In [ ]:
file_data = header+html_table
my_button = SaveFilesButton(file_data)
my_button # This will display the button in the context of Jupyter Notebook